In [9]:
from sklearn import datasets
from sklearn.preprocessing import label_binarize

import random
import numpy as np
import tensorflow as tf

dat = datasets.load_iris()
iris = np.column_stack([dat['data'], dat['target']])
np.random.shuffle(iris)
target_multiclass = label_binarize(iris[:, -1], classes=[0, 1, 2])

print(iris)
print(len(iris))


[[ 6.7  3.3  5.7  2.1  2. ]
 [ 4.8  3.   1.4  0.3  0. ]
 [ 5.8  2.6  4.   1.2  1. ]
 [ 6.9  3.1  4.9  1.5  1. ]
 [ 7.9  3.8  6.4  2.   2. ]
 [ 5.9  3.   4.2  1.5  1. ]
 [ 5.6  2.8  4.9  2.   2. ]
 [ 5.7  2.8  4.5  1.3  1. ]
 [ 5.1  3.7  1.5  0.4  0. ]
 [ 6.3  2.5  5.   1.9  2. ]
 [ 5.6  2.7  4.2  1.3  1. ]
 [ 6.7  3.1  4.7  1.5  1. ]
 [ 4.6  3.6  1.   0.2  0. ]
 [ 6.3  3.3  6.   2.5  2. ]
 [ 6.5  3.   5.2  2.   2. ]
 [ 5.4  3.   4.5  1.5  1. ]
 [ 6.4  3.2  5.3  2.3  2. ]
 [ 6.7  3.   5.   1.7  1. ]
 [ 5.8  2.7  5.1  1.9  2. ]
 [ 5.   3.5  1.6  0.6  0. ]
 [ 7.6  3.   6.6  2.1  2. ]
 [ 6.4  2.8  5.6  2.1  2. ]
 [ 5.7  4.4  1.5  0.4  0. ]
 [ 5.   3.6  1.4  0.2  0. ]
 [ 6.3  2.8  5.1  1.5  2. ]
 [ 7.2  3.6  6.1  2.5  2. ]
 [ 6.4  2.8  5.6  2.2  2. ]
 [ 4.5  2.3  1.3  0.3  0. ]
 [ 5.4  3.7  1.5  0.2  0. ]
 [ 6.3  2.9  5.6  1.8  2. ]
 [ 4.8  3.4  1.6  0.2  0. ]
 [ 5.6  3.   4.5  1.5  1. ]
 [ 5.7  3.8  1.7  0.3  0. ]
 [ 6.4  2.7  5.3  1.9  2. ]
 [ 5.   3.3  1.4  0.2  0. ]
 [ 5.7  2.9  4.2  1.

In [25]:
#network parametrs
n_hidden_1 = 40
n_hidden_2 = 40
n_input = 4
n_classes = 3

# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 20
display_step = 1

# tf Graph input
X = tf.placeholder('float', [None, n_input])
Y = tf.placeholder('float', [None, n_classes])

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Create model
# Hidden fully connected layer with 12 neurons
def perceptron(matrix):
    layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    return tf.matmul(layer_2, weights['out']) + biases['out']

# Construction
logits = perceptron(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initializing  the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(150/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = iris[i:i+batch_size,:4], target_multiclass[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: iris[:,:-1], Y: target_multiclass}))

Epoch: 0001 cost=380.598833357
Epoch: 0002 cost=319.609117780
Epoch: 0003 cost=259.866289411
Epoch: 0004 cost=200.328101022
Epoch: 0005 cost=139.748608180
Epoch: 0006 cost=78.242679051
Epoch: 0007 cost=20.808254514
Epoch: 0008 cost=18.943634987
Epoch: 0009 cost=10.589003290
Epoch: 0010 cost=6.594207764
Epoch: 0011 cost=5.848920277
Epoch: 0012 cost=3.932951076
Epoch: 0013 cost=3.592430728
Epoch: 0014 cost=2.362172314
Epoch: 0015 cost=2.053551640
Epoch: 0016 cost=1.662221585
Epoch: 0017 cost=1.367947153
Epoch: 0018 cost=1.106203641
Epoch: 0019 cost=0.900237381
Epoch: 0020 cost=0.680260573
Epoch: 0021 cost=0.513203123
Epoch: 0022 cost=0.406363457
Epoch: 0023 cost=0.312495568
Epoch: 0024 cost=0.236918356
Epoch: 0025 cost=0.188658418
Epoch: 0026 cost=0.160688098
Epoch: 0027 cost=0.141816156
Epoch: 0028 cost=0.126536415
Epoch: 0029 cost=0.114665846
Epoch: 0030 cost=0.105603279
Optimization Finished!
Accuracy: 0.946667
